In [1]:
# Setup OpenAI with credentials
import openai
from llama_index.agent import OpenAIAgent

openai.api_key = 'sk-key'

In [2]:
from llama_hub.file.sdl.base import SDLReader
from llama_index.tools.ondemand_loader_tool import OnDemandLoaderTool

documentation_tool = OnDemandLoaderTool.from_defaults(
    SDLReader(),
    name="graphql_writer",
    description="""
        The GraphQL schema file is located at './data/github.graphql', this is always the file argument.
        A tool for processing the GitHub GraphQL spec, and writing queries from the documentation.

        You should pass a query_str to this tool in the form of a request to write a GraphQL query.

        Examples:
            file: './data/github.graphql', query_str='Write a graphql query to find unshipped orders'
            file: './data/github.graphql', query_str='Write a graphql query to retrieve the stores products'
            file: './data/github.graphql', query_str='What fields can you retrieve from the orders object'

        """
)


In [3]:
from llama_hub.tools.graphql.base import GraphQLToolSpec

url = 'https://api.github.com/graphql'
headers = {
    'content-type': 'application/json',
    'Authorization': 'bearer your-key'
}

graphql_spec = GraphQLToolSpec(
    url=url,
    headers=headers
)
agent = OpenAIAgent.from_tools(
    [*graphql_spec.to_tool_list(), documentation_tool],
    system_prompt=f"""
    You are a specialized Agent with access to the GitHub GraphQL API.
    Your job is to chat with store owners and help them run GraphQL queries, interpreting the results for the user
 
    You can use graphql_writer to query the schema and assist in writing queries.
    
    If the GraphQL you execute returns an error, either directly fix the query, or directly ask the graphql_writer questions about the schema instead of writing graphql queries.
    Then use that information to write the correct graphql query
    """,
    verbose=True,
    max_function_calls=20
)


In [4]:
print(agent.chat('what kind of objects does this graphql endpoint provide'))

=== Calling Function ===
Calling function: graphql_writer with args: {
  "filename": "./data/github.graphql",
  "query_str": "What objects are available in the GraphQL endpoint?"
}
Got output: 
The objects available in the GraphQL endpoint are: CodeOfConduct, Enterprise, EnterpriseAdministratorInvitation, License, MarketplaceCategory, MarketplaceListing, and Starrable.
The GraphQL endpoint provides the following objects:

1. CodeOfConduct
2. Enterprise
3. EnterpriseAdministratorInvitation
4. License
5. MarketplaceCategory
6. MarketplaceListing
7. Starrable

These objects represent different entities and can be queried and manipulated using GraphQL queries.


In [5]:
print(agent.chat('what license does emptycrown/llama-hub have'))

=== Calling Function ===
Calling function: graphql_request with args: {
  "query": "query License($owner: String!, $repo: String!) {\n  repository(owner: $owner, name: $repo) {\n    licenseInfo {\n      name\n    }\n  }\n}",
  "variables": {
    "owner": "emptycrown",
    "repo": "llama-hub"
  },
  "operation_name": "License"
}
Got output: {"data":{"repository":{"licenseInfo":{"name":"MIT License"}}}}
The "emptycrown/llama-hub" repository has the MIT License.


In [6]:
print(agent.chat('what are the most recent issues and numbers'))

=== Calling Function ===
Calling function: graphql_request with args: {
  "query": "query MostRecentIssues($owner: String!, $repo: String!) {\n  repository(owner: $owner, name: $repo) {\n    issues(last: 5) {\n      nodes {\n        number\n        title\n      }\n    }\n  }\n}",
  "variables": {
    "owner": "emptycrown",
    "repo": "llama-hub"
  },
  "operation_name": "MostRecentIssues"
}
Got output: {"data":{"repository":{"issues":{"nodes":[{"number":419,"title":"Confluence connector reading pagination"},{"number":429,"title":"GoogleSheetsLoader - unable to read credentials.json"},{"number":441,"title":"Can't load AudioTranscriber()"},{"number":451,"title":"Exception: Failed to download https://api.docugami.com/v1preview1/docsets/qdiwmggnlkgw/documents/jletqxsefuge/dgml (status: 404)"},{"number":454,"title":"ERROR:custom_loader:An error occurred while loading data from fileids meta: __init__() takes exactly 1 positional argument (2 given)"}]}}}}
The most recent issues in the "empty

In [7]:
print(agent.chat('tell me more about the googlesheetsloader issue'))

=== Calling Function ===
Calling function: graphql_request with args: {
  "query": "query Issue($owner: String!, $repo: String!, $issueNumber: Int!) {\n  repository(owner: $owner, name: $repo) {\n    issue(number: $issueNumber) {\n      title\n      body\n      createdAt\n      updatedAt\n      author {\n        login\n      }\n    }\n  }\n}",
  "variables": {
    "owner": "emptycrown",
    "repo": "llama-hub",
    "issueNumber": 429
  },
  "operation_name": "Issue"
}
Got output: {"data":{"repository":{"issue":{"title":"GoogleSheetsLoader - unable to read credentials.json","body":"I'm writing a llama-index ingestion class and am using the GoogleSheetsLoader. I set up OAuth and added 'credentials.json' to the directory I run the loader in as stated in the documentation. I'm getting the credentials missing error below:\r\n\r\n```python\r\n  File \"/Users/rishabhshah/Documents/AgentGPT/platform/reworkd_platform/services/llama/llama.py\", line 27, in load_google_sheets\r\n    loaded_docume

In [8]:
print(agent.chat('how could we fix the issue'))

To fix the issue of "GoogleSheetsLoader - unable to read credentials.json", we can try the following steps:

1. Verify the location of the "credentials.json" file: Make sure that the "credentials.json" file is located in the correct directory where the loader is being executed. Double-check the file path and ensure that it is accessible.

2. Check the file permissions: Ensure that the "credentials.json" file has the necessary read permissions. Make sure that the file is not locked or being used by another process.

3. Confirm the file name and format: Verify that the file name is spelled correctly and matches the expected name "credentials.json". Additionally, ensure that the file is in the correct JSON format and does not contain any syntax errors.

4. Validate the OAuth setup: Review the OAuth setup process and ensure that all the required steps have been followed correctly. Check if any additional configuration or authorization steps are needed to access the Google Sheets API.

5. U